## Implementation of AlexNet

### Building the model

In [1]:
import tensorflow as tf
from tensorflow import keras

In [45]:
def buildModel(numClasses, seed=2019):
    
    poolZ = 3
    poolS = 2
    
    inputs1 = keras.layers.Input(shape=(227, 227, 3), dtype=tf.float32)
    
    # conv1: 96 kernels 11x11x3, stride=4, padding=valid
    x = keras.layers.Conv2D(96, 11, strides=(4, 4), padding='valid', activation=None,
                            input_shape=(227, 227, 3), name='conv1')(inputs1)
    #TODO: response normalization?!
    # maxpool1: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu1')(x)
    
    # conv2: 256 kernels 5x5x96, stride=1, padding=same
    x = keras.layers.Conv2D(256, 5, padding='same', activation=None, name='conv2')(x)
    #TODO: response normalization?!
    # maxpool2: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu2')(x)
    
    # conv3: 384 kernels 3x3x256
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv3')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu3')(x)
    
    # conv4: 384 kernels 3x3x384
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv4')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu4')(x)    
    
    # conv5: 256 kernels 3x3x384
    x = keras.layers.Conv2D(256, 3, padding='same', activation=None, name='conv5')(x)
    # maxpool5: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool5')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu5')(x)
    
    # automatic flattening...
    x = keras.layers.Flatten('channels_last', name='flatten')(x)
    
    # fc1: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc1_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x)
    
    # fc2: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc2_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x) 
    
    # fc3: numClasses units
    x = keras.layers.Dense(numClasses, activation='softmax', name='fc3')(x)
    
    outputs = x
    
    return keras.models.Model(inputs = inputs1, outputs = outputs)

### Reading the dataset

In [46]:
import os
import pandas as pd

In [179]:
def readMyDataset(directory, justRead=False, validation=False):
    # make a list of all image files
    #imgDirectory = os.path.join(directory, 'processed', 'img')
    #listFiles = [os.path.abspath(f) for f in os.scandir(imgDirectory)]
    #print("Found {} image files.".format(len(listFiles)))
    
    processedDir = {False: 'processed', True: 'validation_processed'}
    
    #if not validation:
    #    descrFile = os.path.join(directory, 'processed', 'fileLabels.txt')
    #else:
    #    descrFile = os.path.join(directory, 'validation_processed', 'fileLabels.txt')
    descrFile = os.path.join(directory, processedDir[validation], 'fileLabels.txt')
    
    #pathnames = []
    #labelStrings = []
    df = {'pathnames': [], 'labelStrings': []}
    # make a list of all the files we'll process and their labels
    for line in open(descrFile, 'r'):
        #print(line)
        (fileName, label) = line.split(' ')
        # remove quotes and make absolute path
        fileName = os.path.abspath(os.path.join(directory, processedDir[validation], 'img', fileName[1:-1]))
        #print("{} -> {}".format(fileName, label))
        df['pathnames'].append(fileName)
        # remove trailing newline
        df['labelStrings'].append(label[0:-1])
        
    print("Read description of {} files ({} labels).".format(len(df['pathnames']), len(df['labelStrings'])))
    
    #print("ddddd\n{}\ndddddd".format(df['labelStrings'][0]))
    
    df = pd.DataFrame({'pathname': df['pathnames'], 'labelString': df['labelStrings']})
    print("DataFrame: {} rows.".format(len(df.index)))
    
    dummyLabels = pd.get_dummies(df['labelString'], prefix='', prefix_sep='')
    #print(dummyLabels)
    numClasses = len(dummyLabels.columns)
    classes = list(dummyLabels.columns.values)
    print("number of classes: {}".format(numClasses))
    
    dset = tf.data.Dataset.from_tensor_slices((df['pathname'], dummyLabels))
    
    # function to read image data
    def _howToRead(pathname, label):
        imageData = tf.cast(tf.image.resize_images(
            tf.image.decode_png(tf.read_file(pathname), channels=3), [227, 227]), tf.float32)
        return (imageData, label)
    
    def _howToNormalize(img, label):
        return (img / 255.0, label)
    
    def _howToAugRot90(img, label):
        return (tf.image.rot90(img, k=1), label)
    
    def _howToAugRot180(img, label):
        return (tf.image.rot90(img, k=2), label)
    
    def _howToAugRot270(img, label):
        return (tf.image.rot90(img, k=3), label)
    
    def _howToRandomModify(img, label):
        img = tf.image.random_hue(img, 0.1)
        img = tf.image.random_brightness(img, 0.2)
        return (img, label)
    
    def _howToAddRandomNoise(img, label):
        img = img + tf.random.uniform(img.shape, -40, 40)
        return (img, label)
    
    dset = dset.map(_howToRead)
    
    if justRead:
        return (dset.map(_howToNormalize), numClasses, classes)
    
    dsetRandomModified = dset.map(_howToRandomModify)
    #dset = dset.map(_howToNormalize).concatenate(dsetRandomModified.map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot90).map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot180).map(_howToNormalize)).concatenate(
    #    dsetRandomModified.map(_howToAugRot270).map(_howToNormalize))
    
    dset = dset.concatenate(dsetRandomModified).concatenate(
        dsetRandomModified.map(_howToAugRot90)).concatenate(
        dsetRandomModified.map(_howToAugRot180)).concatenate(
        dsetRandomModified.map(_howToAugRot270))
    dset = dset.concatenate(dset.map(_howToAddRandomNoise))
    dset = dset.map(_howToNormalize)
    
    return (dset, numClasses, classes)
    
    #print("dummies:\n{}".format(dummyLabels))

In [48]:
dataset, numClasses, classes = readMyDataset("/home/ivan/datasets/bilbao2019")

Read description of 4018 files (4018 labels).
DataFrame: 4018 rows.
number of classes: 17


In [49]:
print("{}".format(dataset))

<DatasetV1Adapter shapes: ((227, 227, 3), (17,)), types: (tf.float32, tf.uint8)>


In [50]:
print("Classes: {}".format(classes))

Classes: ['ad', 'bench', 'container', 'door', 'extension', 'gate', 'logo', 'manhole', 'socket', 'staircase', 'switch', 'tap', 'textad', 'toilet', 'urinal', 'urn', 'washbasin']


In [39]:
# just to check if everything is ok

iterator = dataset.make_one_shot_iterator()
nextElement = iterator.get_next()

with tf.Session() as sess:
    cnt = 0
    while True:
        try:
            imgbin, label = sess.run(nextElement)
            #print("{}\n".format(imgbin))
            print("Label # {}: {}".format(cnt, label))
            cnt += 1
        except tf.errors.OutOfRangeError:
            break
    

Label # 0: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 2: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 3: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 4: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 5: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 6: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 7: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 8: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 9: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 10: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 11: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 12: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 13: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 14: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 15: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 16: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 17: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 18: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 19: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 20: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
La

Label # 190: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 191: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 192: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 193: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 194: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 195: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 196: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 197: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 198: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 199: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 200: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 201: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 202: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 203: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 204: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 205: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 206: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 207: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 208: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 209: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 210: [0 1 0 

Label # 399: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 400: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 401: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 402: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 403: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 404: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 405: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 406: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 407: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 408: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 409: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 410: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 411: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 412: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 413: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 414: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 415: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 416: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 417: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 418: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 419: [0 1 0 

Label # 578: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 579: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 580: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 581: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 582: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 583: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 584: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 585: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 586: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 587: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 588: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 589: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 590: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 591: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 592: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 593: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 594: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 595: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 596: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 597: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 598: [0 0 0 

Label # 782: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 783: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 784: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 785: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 786: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 787: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 788: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 789: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 790: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 791: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 792: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 793: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 794: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 795: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 796: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 797: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 798: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 799: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 800: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 801: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 802: [0 0 0 

Label # 1001: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1002: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1003: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1004: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1005: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1006: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1007: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1008: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1009: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1010: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1011: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1012: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1013: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1014: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1015: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1016: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1017: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1018: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1019: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1020: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


Label # 1206: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1207: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1208: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1209: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1210: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1211: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1212: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1213: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1214: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1215: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1216: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1217: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1218: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1219: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1220: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1221: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1222: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1223: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1224: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 1225: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


Label # 1378: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1379: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1380: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1381: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1382: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1383: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1384: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1385: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1386: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1387: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1388: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1389: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1390: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1391: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1392: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1393: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1394: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1395: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1396: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1397: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


Label # 1588: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1589: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1590: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1591: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1592: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1593: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1594: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1595: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1596: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1597: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1598: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1599: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1600: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1601: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1602: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1603: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1604: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1605: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1606: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 1607: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


KeyboardInterrupt: 

### Inputs and outputs

In [51]:
dataset = dataset.shuffle(buffer_size=3000).repeat().batch(32).prefetch(1)

In [52]:
# just to check if everything is ok

#iterator = dataset.make_one_shot_iterator()
#nextElement = iterator.get_next()
#
#with tf.Session() as sess:
#    cnt = 0
#    while True:
#        try:
#            imgbin, label = sess.run(nextElement)
#            #print("{}\n".format(imgbin))
#            print("Label # {}: {}".format(cnt, label))
#            cnt += 1
#        except tf.errors.OutOfRangeError:
#            break

In [53]:
iterator = dataset.make_one_shot_iterator()
inputs, targets = iterator.get_next()
#modelInput = tf.keras.Input(tensor=inputs)
##modelInput = tf.keras.Input(tensor=dataset)

In [54]:
#numClasses = 17
model = buildModel(numClasses)

In [55]:
targets = tf.cast(targets, tf.float32)

In [57]:
#model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy',
#             metrics=['accuracy'])#, 
             #target_tensors=targets)
model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy',
             metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_v1_14 (B (None, 27, 27, 96)        384       
_________________________________________________________________
relu1 (Activation)           (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 13, 13, 256)       0         
__________

In [150]:
#model.fit(epochs=10, steps_per_epoch=50)
#model.fit(epochs=2, steps_per_epoch=50)
#model.fit(epochs=1, steps_per_epoch=5)
model.fit(inputs, targets, epochs=1, steps_per_epoch=5)
#model.fit(inputs, targets, epochs=4, steps_per_epoch=1024)

5/5 [==============================] - 20s 4s/step - loss: 0.0042 - acc: 1.0000


### Save model and weights

In [18]:
modelJson = model.to_json()
with open("alexnet_bilbao2019_new2.json", "w") as jsonFile:
    jsonFile.write(modelJson)

In [36]:
#model.save_weights("alexnet_bilbao2019_weights.h5")
model.save_weights("alexnet_bilbao2019_weights_new3.h5")

### Recover weights

In [58]:
model.load_weights("alexnet_bilbao2019_weights_new3.h5")

### Try with some photos

In [68]:
import numpy as np


def tryOnPhoto(model, photoPathname):
    photo = tf.image.decode_jpeg(tf.read_file(photoPathname), channels=3)
    photo = tf.image.resize_images(photo, (227, 227)) / 255.0
    
    modelInput = None
    photoBin = None
    with tf.Session() as sess:
        photoBin = sess.run(photo)
    
    photoBin = np.expand_dims(photoBin, axis=0)
    print(photoBin.shape)
    
    photoBin = tf.constant((photoBin), shape = (1, 227, 227, 3))
    print(photoBin)
    photoBin = tf.data.Dataset.from_tensor_slices(photoBin).batch(1).repeat()
    print(photoBin)
    iterator = photoBin.make_one_shot_iterator()    
    inputs = iterator.get_next()
    #modelInput = keras.layers.Input(tensor=inputs, shape=(1, 227, 227, 3))
        
    print(modelInput)
    return model.predict(inputs, batch_size=1, steps=1, verbose=1)
    #return model.predict(photoBin)

In [316]:
#testRes = tryOnPhoto(model, "the-fake-downing-street-10-door.jpg")
#testRes = tryOnPhoto(model, "blue-door-london.jpg")
#testRes = tryOnPhoto(model, "e66858cd-2fcc-41e3-8e4f-5b2b7eb71705.jpeg")
#testRes = tryOnPhoto(model, "contenedores_basura-6-1-1068x712.jpg")
#testRes = tryOnPhoto(model, "1506704365_421985_1506704486_noticia_normal.jpg")
#testRes = tryOnPhoto(model, "staircase_sf1.jpeg")
#testRes = tryOnPhoto(model, "/home/ivan/datasets/bilbao2019/raw/logo/IMG_20190403_094234.jpg")
#testRes = tryOnPhoto(model, "peligro-ratas.jpg")
#testRes = tryOnPhoto(model, "perritoslindos.jpg")
#testRes = tryOnPhoto(model, "contenedores_cerca_auto_basura.jpg_501420591.jpg")
#testRes = tryOnPhoto(model, "cp20f08733531_829923_manual.jpg")
#testRes = tryOnPhoto(model, "5b523b8d018a2.jpeg")
#testRes = tryOnPhoto(model, "Joe-s-Doors-256.jpg")
#testRes = tryOnPhoto(model, "door1.jpg")
#testRes = tryOnPhoto(model, "door2.jpg")
#testRes = tryOnPhoto(model, "door3.jpg")
#testRes = tryOnPhoto(model, "banco2.jpeg")
#testRes = tryOnPhoto(model, "banho1.jpg")
#testRes = tryOnPhoto(model, "banho2.jpg")
#testRes = tryOnPhoto(model, "banho3.jpg")
#inputs, targets = iterator.get_next()
#testRes = model.predict(inputs, batch_size=1, steps=1)

#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/container/IMG_20190408_190520.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/ad/IMG_20190408_184101.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/urn/IMG_20190408_204752.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/toilet/IMG_20190408_114148.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/washbasin/IMG_20190408_113930.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/door/IMG_20190408_205937.jpg")
#testRes = tryOnPhoto(model, "/home/ivan/Pictures/photodata/test/textad/IMG_20190408_204328.jpg")
testRes = tryOnPhoto(model, "/home/ivan/datasets/bilbao2019/validation_processed/img/00063.png")

(1, 227, 227, 3)
Tensor("Const_88:0", shape=(1, 227, 227, 3), dtype=float32)
<DatasetV1Adapter shapes: (?, 227, 227, 3), types: tf.float32>
None
1/1 [==============================] - 1s 542ms/step


In [317]:
print("class: {}, score: {}".format(classes[np.argmax(testRes[0])], np.max(testRes[0])))

class: tap, score: 1.0


In [241]:
print(testRes)

[[2.5020108e-01 6.4326502e-02 2.1231648e-01 7.3950261e-02 3.5298878e-04
  1.6334089e-07 2.9335751e-03 1.0758079e-06 2.4129769e-04 4.4570947e-07
  9.5646692e-06 4.0194124e-02 2.8447519e-05 1.2371442e-03 5.7504518e-04
  3.3923158e-01 1.4400198e-02]]


In [47]:
classes

['ad',
 'bench',
 'container',
 'door',
 'extension',
 'gate',
 'logo',
 'manhole',
 'socket',
 'staircase',
 'switch',
 'tap',
 'textad',
 'toilet',
 'urinal',
 'urn',
 'washbasin']

In [186]:
datasetValid, numClassesValid, classesValid = readMyDataset("/home/ivan/datasets/bilbao2019", 
                                                            justRead=True, validation=True)
datasetValid = datasetValid.batch(32)

Read description of 64 files (64 labels).
DataFrame: 64 rows.
number of classes: 17


In [187]:
iteratorValid = datasetValid.make_one_shot_iterator()
inputsValid, targetsValid = iteratorValid.get_next()

In [188]:
validOutput = model.evaluate(x=inputsValid, y=targetsValid, verbose=1, steps=2)

2/2 [==============================] - 3s 1s/step - loss: 0.7982 - acc: 0.8438


In [153]:
print("dataset:\n{}\ndatasetValid:\n{}".format(dataset, datasetValid))

dataset:
<DatasetV1Adapter shapes: ((?, 227, 227, 3), (?, 17)), types: (tf.float32, tf.uint8)>
datasetValid:
<DatasetV1Adapter shapes: ((?, 227, 227, 3), (?, 17)), types: (tf.float32, tf.uint8)>


In [154]:
model.metrics_names

['loss', 'acc']

In [302]:
validOutput

[0.7981506288051605, 0.84375]

In [65]:
classesValid

['ad',
 'bench',
 'container',
 'door',
 'extension',
 'gate',
 'logo',
 'manhole',
 'socket',
 'staircase',
 'switch',
 'tap',
 'textad',
 'toilet',
 'urinal',
 'urn',
 'washbasin']

In [66]:
classes

['ad',
 'bench',
 'container',
 'door',
 'extension',
 'gate',
 'logo',
 'manhole',
 'socket',
 'staircase',
 'switch',
 'tap',
 'textad',
 'toilet',
 'urinal',
 'urn',
 'washbasin']

In [183]:
iteratorValid = datasetValid.make_one_shot_iterator()
inputsValid, targetsValid = iteratorValid.get_next()

In [184]:
predictOnValid = model.predict(inputsValid, steps=2)

In [185]:
predictOnValid

array([[6.75510869e-10, 9.99999166e-01, 2.26410904e-12, 9.72524172e-10,
        8.04977699e-12, 2.50959388e-12, 4.50264088e-12, 4.00723117e-08,
        5.13050262e-11, 2.62075190e-07, 1.47580117e-08, 3.82445791e-07,
        1.56631319e-10, 6.63842386e-08, 1.07360468e-10, 5.16857446e-10,
        5.70957621e-11],
       [4.93493599e-06, 9.99938726e-01, 3.04301921e-07, 4.73562341e-06,
        3.17734644e-10, 5.65066771e-10, 3.13810145e-10, 4.95032873e-05,
        1.08161419e-06, 3.56593949e-10, 1.87921323e-08, 7.97222341e-08,
        2.82657092e-11, 1.28070861e-11, 2.37495544e-11, 6.83606743e-07,
        3.22574523e-10],
       [1.79995720e-06, 1.65250908e-15, 1.82906153e-13, 5.72749605e-06,
        1.62190191e-14, 1.78351070e-06, 1.12276310e-09, 1.21056255e-13,
        4.20206574e-14, 6.06661943e-11, 7.16519780e-13, 4.68374146e-13,
        9.99990702e-01, 4.71682762e-11, 1.47172743e-11, 1.53552290e-12,
        2.55538334e-11],
       [4.55906428e-03, 7.59570301e-02, 5.85244067e-08, 1.861

In [175]:
iteratorValid = datasetValid.make_one_shot_iterator()
inputsValid, targetsValid = iteratorValid.get_next()

with tf.Session() as sess:
    while True:
        try:
            inpVal, tarVal = sess.run([inputsValid, targetsValid])
            print("{}".format(tarVal))
        except tf.errors.OutOfRangeError:
            break

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 